## Explorinig the Data

**We will download the images and then trainthe model by using CNNs and ResNets.**

In [1]:
import os 
import torch 
import tarfile
import torchvision
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [2]:
# Download
data_url = "http://files.fast.ai/data/wt103_ids.tgz"
download_url(data_url,'.')

Using downloaded and verified file: .\wt103_ids.tgz


In [3]:
data_dir = 'archive (3)\cifar10\cifar10'
print(os.listdir(data_dir))

['test', 'train']


In [4]:
airplane_files = os.listdir('archive (3)\\cifar10\\cifar10\\train\\airplane')
print('No of test examples for airplane:', len(airplane_files))
print(airplane_files[:5])

No of test examples for airplane: 5000
['0001.png', '0002.png', '0003.png', '0004.png', '0005.png']


In [8]:
dataset = ImageFolder(data_dir+'/train', transform=ToTensor())

Look at the sample element from the training dataset . Each element is given as a tuple, containing image tensor and a label.


In [9]:
img, lbl = dataset[0]
print(img.shape,lbl)
img

torch.Size([3, 32, 32]) 0


tensor([[[0.7922, 0.7922, 0.8000,  ..., 0.8118, 0.8039, 0.7961],
         [0.8078, 0.8078, 0.8118,  ..., 0.8235, 0.8157, 0.8078],
         [0.8235, 0.8275, 0.8314,  ..., 0.8392, 0.8314, 0.8235],
         ...,
         [0.8549, 0.8235, 0.7608,  ..., 0.9529, 0.9569, 0.9529],
         [0.8588, 0.8510, 0.8471,  ..., 0.9451, 0.9451, 0.9451],
         [0.8510, 0.8471, 0.8510,  ..., 0.9373, 0.9373, 0.9412]],

        [[0.8000, 0.8000, 0.8078,  ..., 0.8157, 0.8078, 0.8000],
         [0.8157, 0.8157, 0.8196,  ..., 0.8275, 0.8196, 0.8118],
         [0.8314, 0.8353, 0.8392,  ..., 0.8392, 0.8353, 0.8275],
         ...,
         [0.8510, 0.8196, 0.7608,  ..., 0.9490, 0.9490, 0.9529],
         [0.8549, 0.8471, 0.8471,  ..., 0.9412, 0.9412, 0.9412],
         [0.8471, 0.8431, 0.8471,  ..., 0.9333, 0.9333, 0.9333]],

        [[0.7804, 0.7804, 0.7882,  ..., 0.7843, 0.7804, 0.7765],
         [0.7961, 0.7961, 0.8000,  ..., 0.8039, 0.7961, 0.7882],
         [0.8118, 0.8157, 0.8235,  ..., 0.8235, 0.8157, 0.

In [10]:
## Plotting Graph
import matplotlib.pyplot as plt
def show_sample(img, lbl):
    print('label: ', dataset.classes[lbl],"('+str(lbl)+)')")
    plt.imshow(img.permute(1,2,0))

In [ ]:
show_sample(*dataset[0])

label:  airplane ('+str(lbl)+)')


In [12]:
import numpy as np
def split_indices(n, val_pct=0.1, seed=99):
    # Determinr size of validation
    n_val = int(val_pct*n)
    # set random seed
    np.random.seed(seed)
    idx = np.random.permutation(n)
    # pick first n_val indices for validation set
    return idx[n_val:], idx[:n_val]

In [13]:
val_pct = 0.2
rand_seed = 42
train_indices, val_indices = split_indices(len(dataset), val_pct, rand_seed)
print(len(train_indices),len(val_indices))
print('Sample validation indices:', val_indices[:10])

40000 10000
Sample validation indices: [33553  9427   199 12447 39489 42724 10822 49498  4144 36958]


In [14]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader 
batch_size = 100

In [15]:
# Trianing sampler and data loader
train_sampler = SubsetRandomSampler(train_indices)
train_dl = DataLoader(dataset, batch_size, sampler=train_sampler)

In [16]:
# Validation sampler and Dataloader
val_sampler = SubsetRandomSampler(val_indices)
val_dl = DataLoader(dataset, batch_size, sampler=val_sampler)

In [17]:
from torchvision.utils import make_grid

def show_batch()

SyntaxError: invalid syntax (3087617693.py, line 3)

In [19]:
import torch.nn as nn
import torch.nn.functional as F

In [20]:
simple_model = nn.Sequential(
                nn.Conv2d(3,8, kernel_size=3, stride=1, padding=1),
                nn.MaxPool2d(2,2))

In [22]:
for img, lbl in train_dl:
    print('images.shape', img.shape)
    out = simple_model(img)
    print('out.shape', out.shape)
    break


images.shape torch.Size([100, 3, 32, 32])
out.shape torch.Size([100, 8, 16, 16])


In [25]:
model = nn.Sequential(
        nn.Conv2d(3,16,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),# batch_size = 16 x 16 x 16
    
        nn.Conv2d(16,16,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
    
    
        nn.Conv2d(16,16,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
    
    
        nn.Conv2d(16,16,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
    
    
        nn.Conv2d(16,16,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
    
       
        nn.Flatten(),
        nn.Linear(16,10))
         
    
        

In [26]:
model[3].weight.shape

torch.Size([16, 16, 3, 3])

## Training the Model


In [29]:
def loss_batch(model ,loss_func, xb,yb, opt=None, metric = None):
    # Generate predictions
    preds = model(xb)
    loss = loss_func(preds, yb)
    
    if opt is not None:
        # Compute gradients
        loss.backward()
        opt.step()
        #reset
        opt.zero_grad()
        
    metric_result = None
    if metric is not None:
        # compute metric
        metric_result = metric(preds, yb)
        
    return loss.item(), len(xb), metric_result

In [8]:
def fit(epochs, model, loss_fn , train_dl,valid_dl,
    opt_fn = None, lr = None, metric = None):
    
    # optimizer 
    if opt_fn is None:
        opt_fn = torch.optim.SGD
        opt = opt_fn(model.parameters(), lr = lr)
        
    for epoch in range(epochs):
        # Training
        model.train()
        for xb, yb in train_dl:
            train_loss,_,_ = loss_batch(model,loss_fn, xb,yb,opt)
            
        # Evaluation
        model.eval()
        res = evaluate(model, loss_fn, valid_dl, metric)
        val_loss, total, val_metric = res
        
        # record loss
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_metrics.append(val_loss)
        
        # print progress
        if metric is None:
            print('Epoch [{}/{}], train_loss: {:4f}, val_loss: {:4f}'.format(epoch+1, epochs, train_loss, val_loss))
            
        else:
            print('Epoch [{}/{}], train_loss: {:4f}, val_loss: {:4f}, val_{}: {:.4f}'.format(epoch+1, epochs, train_loss, val_loss,
                                                                                            metric.__name, val_metric))
            
    return train_losses, val_losses, val_metrics
            
            
     

In [9]:
def accuracy(outputs, lbl):
    _, preds = torch.max(outputs, dim =1)
    return torch.sum(preds == lbl).items()/ len(preds)


In [12]:
## evaluate model

In [16]:
def plot_metric(metruc_values):
    plt.plot(metric_values,'-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs No. of epochs')

In [15]:
# plot by yourself